#### CSCE 670 :: Information Storage and Retrieval :: Texas A&M University :: Spring 2018


# Homework 2:  Link Analysis -- HITS + SEO

### 100 points [5% of your final grade]

### Due: Sunday, February 25, 2018 by 11:59pm

*Goals of this homework:* Explore real-world challenges of building a graph (in this case, from tweets), implement and test HITS algortihm over this graph, and investigate factors that impact a page's rank on Google and Bing.

*Submission Instructions:* To submit your homework, rename this notebook as YOUR_UIN_hw2.ipynb. Submit this notebook via ecampus. Your notebook should be completely self-contained, with the results visible in the notebook. 

*Late submission policy:* For this homework, you may use up to three of your late days, meaning that no submissions will be accepted after Wednesday, February 28, 2018 at 11:59pm.

# Part 1: HITS (70 points)

## A re-Tweet Graph

In this assignment, we're going to adapt the classic HITS approach to allow us to find not the most authoritative web pages, but rather to find significant Twitter users. So, instead of viewing the world as web pages with hyperlinks (where pages = nodes, hyperlinks = edges), we're going to construct a graph of Twitter users and their retweets of other Twitter users (so user = node, retweet of another user = edge). Over this Twitter-user graph, we can apply the HITS approach to order the users by their hub-ness and their authority-ness.

Here is a toy example. Suppose you are given the following four retweets:

* **userID**: diane, **text**: "RT ", **sourceID**: bob
* **userID**: charlie, **text**: "RT Welcome", **sourceID**: alice
* **userID**: bob, **text**: "RT Hi ", **sourceID**: diane
* **userID**: alice, **text**: "RT Howdy!", **sourceID**: parisa

There are four short tweets retweeted by four users. The retweet between users form a directed graph with five nodes and four edges. E.g., the "diane" node has a directed edge to the "bob" node.

You should build a graph by parsing the tweets in the file we provide called *HITS.json*.

**Notes:**

* You may see some weird characters in the content of tweets, just ignore them. 
* The edges are weighted and directed. If Bob retweets Alice's tweets 10 times, there is an edge from Bob to Alice with weight 10, but there is not an edge from Alice to Bob.
* If a user retweets herself, ignore it.
* Correctly parsing screen_name in a tweet is error-prone. Use the id of the user (this is the user who is re-tweeting) and the id of the user in the retweeted_status field (this is the user who is being re-tweeted; that is, this user created the original tweet).
* Later you will need to implement the HITS algorithm on the graph you build here.


In [42]:
# building the graph by parsing HITS.json, this gives us A
# The retweets are provided in the form of json data, we need to extract the userId and sourceId.
# userId refers to the user who retweets and sourceId refers to the user who posted the original tweet
import json
import numpy as np

tweet_data = 'HITS.json'
# Read the file to figure out the size of the graph, this is needed to initialize the adjacency matrix
with open(tweet_data) as file:
    # Create a set of all users, this automatically takes care of the duplicates
    setUsers = set()
    for line in file:
        data = json.loads(line)
        userId = data['user']['id']
        sourceId = data['retweeted_status']['user']['id']
        # In the sense of HITS, userId is the hub which points to sourceId which is the authority.
        # The graph can be represented as a adjacency matrix consisting of all the edges.
        if userId not in setUsers:
            setUsers.add(userId)
        if sourceId not in setUsers:
            setUsers.add(sourceId)
# Number of elements in setUsers is equal to the number of nodes in the graph
numUsers = len(setUsers)
# A is a square matrix of size equal to the number of unique users
A = np.matrix(np.zeros(shape=(numUsers,numUsers)))
# create a list mapping indices in the adjacency matrix to users 
idx2User = list(setUsers)
# create a dictionary mapping users to indices in the adjacency matrix
user2Idx = {}
for (idx,user) in enumerate(idx2User):
    user2Idx[user] = idx    
    
# We now have our graph, we just need to add the edges. So, read file again to fill up the matrix
with open(tweet_data) as file:
    for line in file:
        data = json.loads(line)
        userId = data['user']['id']
        sourceId = data['retweeted_status']['user']['id']
        # Retweeting is like linking to a web page, so add an edge from userId to sourceId
        firstNode = user2Idx[userId]
        secondNode = user2Idx[sourceId]
        A[firstNode,secondNode] += 1


We will not check the correctness of your graph. However, this will affect the HITS results later.

## HITS Implementation

Your program will return the top 10 users with highest hub and authority scores. The **output** should be like:

Hub Scores

* user1 - score1
* user2 - score2
* ...
* user10 - score10

Authority Scores

* user1 - score1
* user2 - score2
* ...
* user10 - score10

You should follow these **rules**:

* Assume all nodes start out with equal scores.
* It is up to you to decide when to terminate the HITS calculation.
* There are HITS implementations out there on the web. Remember, your code should be **your own**.


**Hints**:
* If you're using the matrix style approach, you should use [numpy.matrix](https://docs.scipy.org/doc/numpy/reference/generated/numpy.matrix.html).
* Scipy is built on top of Numpy and has support for sparse matrices. You most likely will not need to use Scipy unless you'd like to try out their sparse matrices.
* If you choose to use Numpy (and Scipy), please make sure your Anaconda environment include their latest versions.
* Test your parsing and HITS calculations using a handful of tweets, before moving on to the entire file we provide.
* We will evaluate the user ranks you provide as well as the quality of your code. So make sure that your code is clear and readable.

In [44]:
# h and a are the hub scores and authority scores
# h = Aa, a = A'h, for 100 iterations

def HITS(A):
    numUsers = A.shape[1]
    A_trans = A.transpose()
    # Initialize the hub scores and authority scores for each user
    h = np.matrix(np.ones(shape=(numUsers,1)))
    h_old = np.matrix(np.ones(shape=(numUsers,1)))
    a = np.matrix(np.ones(shape=(numUsers,1)))
    a_old = np.matrix(np.ones(shape=(numUsers,1)))  
    
    for _ in range(100):
        h = A.dot(a_old)
        a = A_trans.dot(h_old)
        # Normalize values to prevent them from exploding!
        h_normalized = h/np.linalg.norm(h)
        a_normalized = a/np.linalg.norm(a)
        h_old = h_normalized
        a_old = a_normalized
    print("Top 10 hubs:")
    for i in range(10):
        maxNodeIdx = np.argmax(h_normalized)
        maxNodeVal = h_normalized[maxNodeIdx,0]
        print('{} : {} - {}'.format(i+1, idx2User[maxNodeIdx], maxNodeVal))
        # Set value to 0 once the value is returned, so that the highest score is not returned again
        h_normalized[maxNodeIdx,0] = 0
     
    print("\nTop 10 authorities:")
    for i in range(10):
        maxNodeIdx = np.argmax(a_normalized)
        maxNodeVal = a_normalized[maxNodeIdx,0]
        print('{} : {} - {}'.format(i+1, idx2User[maxNodeIdx], maxNodeVal))
        a_normalized[maxNodeIdx,0] = 0

HITS(A)

Top 10 hubs:
1 : 3068706044 - 0.623118589402
2 : 3093940760 - 0.29616065505
3 : 2194518394 - 0.259872557878
4 : 2862783698 - 0.202585650131
5 : 3092183276 - 0.170518444546
6 : 3029724797 - 0.166989841914
7 : 2990704188 - 0.147733304378
8 : 3001500121 - 0.144826539971
9 : 3086921438 - 0.129150711052
10 : 3042686360 - 0.12526337271

Top 10 authorities:
1 : 3042570996 - 0.544624795117
2 : 3065514742 - 0.493216581263
3 : 1638625987 - 0.443930691851
4 : 3077733683 - 0.286591301511
5 : 3039321886 - 0.224333549101
6 : 3077695572 - 0.121884153741
7 : 3019659587 - 0.113227816932
8 : 1358345766 - 0.0980315982566
9 : 3061155846 - 0.0939853064596
10 : 3092580049 - 0.0936792602878


# Part 2: Search Engine Optimization (30 + 5 points)

For this part, your goal is to put on your "[search engine optimization](https://en.wikipedia.org/wiki/Search_engine_optimization)" hat. Your job is to create a webpage that scores highest for the query: **kbeznak parmatonic** --- two terms, lower case, no quote. As of today (Feb 16, 2018), there are no hits for this query on either Google or Bing. Based on our discussions of search engine ranking algorithms, you know that several factors may impact a page's rank. Your goal is to use this knowledge to promote your own page to the top of the list.

What we're doing here is a form of [SEO contest](https://en.wikipedia.org/wiki/SEO_contest). While you have great latitude in how you approach this problem, you are not allowed to engage in any unethical or illegal behavior. Please read the discussion of "white hat" versus "black hat" SEO over at [Wikipedia](https://en.wikipedia.org/wiki/Search_engine_optimization#White_hat_versus_black_hat_techniques).


**Rules of the game:**

* Somewhere in the page (possibly in the non-viewable source html) you must include your name or some other way for us to identify you (e.g., your NetID, but not the UIN!).
* Your target page may only be a TAMU student page, a page on your own webserver, a page on a standard blog platform (e.g., wordpress), or some other primarily user-controlled page
* Your target page CAN NOT be a twitter account, a facebook page, a Yahoo Answers or similar page
* No wikipedia vandalism
* No yahoo/wiki answers questions
* No comment spamming of blogs
* If you have concerns/questions/clarifications, please post on Piazza and we will discuss

For your homework turnin for this part, you should provide us the URL of your target page and a brief discussion (2-4 paragraphs) of the strategies you are using. We will issue the query and check the rankings at some undetermined time in the next couple of weeks. You might guess that major search engines take some time to discover and integrate new pages: if I were you, I'd get a target page up immediately.

**Grading:**

* 5 points for providing a valid URL
* 20 points for a well-reasoned discussion of your strategy
* 5 points for your page appearing in the search results by Google or Bing (no matter how is the ranking)

** Bonus: **
* 1 point for your page appearing in the top-20 on Google or Bing
* 1 more point for your page appearing in the top-10 on Google or Bing
* 1 more point for your page appearing in the top-5 on Google or Bing
* 2 more points for your page being ranked first by Google or Bing. And, a vigorous announcement in class, and a high-five for having the top result!

What's the URL of your page?


https://vorzawk.github.io/kbeznak-parmatonic.html

What's your strategy? (2-4 paragraphs)


My main strategy was to make an awesome webpage since I am not very good at making up stories. Besides, the Google SEO Guide suggests that user experience is valued very highly when it comes to site rankings. So, I wrote about what SEO means and how kbeznak parmatonic ties into a class on Information retieval. I also optimized my website for mobile viewing using RWD concepts. Basically, I tried to make it as interesting and informational as possible while only using real information.

The key idea in PageRank is that backlinks are basically endorsements to the webpage. So, I tried to add links to my webpage in as many webpages I had access to i.e my tamu webpage, linkedin account, a quora answer and other places.

I also signed up for webmaster accounts in both Bing and Google Analytics, verified my website and explicitly submitted my webpage for indexing. I also added a sitemap so that the crawler knows its way around my site.